In [1]:
stkprm=100 
stkvol=1
ngdesired=100

from opentrons import protocol_api

import pandas
import numpy as np
import os

In [2]:
os.chdir("C:/Users/jonbr/Documents/Github/opentrons/Golden_Gate/Part2_Assembly_Cam")
os.getcwd()
fragments = pandas.read_csv('fragments.csv')
fragments

,Frag,Conc,basepairs
0,0,60,1104
1,1,60,2072
2,2,60,6447


In [3]:
backbone_length=fragments["basepairs"].max()
backbone_length

6447

In [5]:
for i, row in fragments.iterrows():
    fragments.loc[i,"equimolar ratio"]=fragments.loc[i,"basepairs"]/backbone_length
    fragments.loc[i,"inverse of conc"]=1/fragments.loc[i,"Conc"]
    fragments.loc[i,"initial required amount"]=ngdesired*fragments.loc[i,"equimolar ratio"]*fragments.loc[i,"inverse of conc"]
    
    if fragments.loc[i,"initial required amount"] > 1: 
        fragments.loc[i,"H20 to add to 1uL of fragment"] = np.nan
    if fragments.loc[i,"initial required amount"] < 1:
        fragments.loc[i,"H20 to add to 1uL of fragment"]=(2/fragments.loc[i,"initial required amount"])-1
    #if fragments.loc[i,"parts H20 to one part fragment"] < 1:
        #fragments.loc[i,"parts H20 to one part fragment"] = 1-fragments.loc[i,"initial required amount"]
    
    
    fragments.loc[i,"new Conc"] = fragments.loc[i,"Conc"]/(fragments.loc[i,"H20 to add to 1uL of fragment"]+1)
    fragments.loc[i,"new required amount"] = ngdesired*fragments.loc[i,"equimolar ratio"]*(1/fragments.loc[i,"new Conc"])
    
    fragments.loc[i,"final amount to add"] = ''
    if fragments.loc[i,"initial required amount"] > 1:
        fragments.loc[i,"final amount to add"] = fragments.loc[i,"initial required amount"]
    else:
        fragments.loc[i,"final amount to add"] = fragments.loc[i, "new required amount"]
    fragments.loc[i,"final dilution location"] = ''
fragments
  

,Frag,Conc,basepairs,equimolar ratio,inverse of conc,initial required amount,H20 to add to 1uL of fragment,new Conc,new required amount,final amount to add,final dilution location
0,0,60,1104,0.171242,0.016667,0.285404,6.007609,8.562122,2.0,,
1,1,60,2072,0.321390,0.016667,0.535650,2.733784,16.069490,2.0,,
2,2,60,6447,1.000000,0.016667,1.666667,NaN,NaN,NaN,,


In [8]:
index = fragments.index
number_of_rows = len(index)
number_of_rows

3

In [ ]:
#amountH2O= 11.35-
#number_of_rows

In [14]:
def run(protocol: protocol_api.ProtocolContext):

    right = protocol.load_instrument(
        'p10_single', 'right', tip_rack=tiprack1)
    left = protocol.load_instrument(
        'p300_single', 'right', tip_rack=tiprack2)
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_10ul', '5')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_200ul', '6')
    plate = protocol.load_labware('corning_24_wellplate_3.4ml_flat', '1')
    tc_mod = protocol.load_module('Thermocycler Module')
    platetc = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    GGmix= platetc['A1']
    T4 = plate['A1']
    BSA = plate ['A2']
    BSAI = plate ['A3']
    Lig = plate ['A4']
    H20 = plate ['A5']
    T4BSAmix = plate ['C6']
    frag1 = plate ['B1']
    frag2 = plate['B2']
    if number_of_rows > 2:
        frag3 = plate['B3']
    if number_of_rows > 3:
        frag4 = plate['B4']
    if number_of_rows > 4:
        frag5 = plate['B5']
    if number_of_rows > 5:
        frag6 = plate['B6']
    frag1dil= plate['D1']
    frag2dil= plate['D2']
    if number_of_rows > 2:
        frag3dil = plate['D3']
    if number_of_rows > 3:
        frag4dil = plate['D4']
    if number_of_rows > 4:
        frag5dil = plate['D5']
    if number_of_rows > 5:
        frag6dil = plate['D6']
    #dilution
    
    right.pick_up_tip()
    right.aspirate(1,frag1)
    right.dispense(1,frag1dil)
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(fragments.loc[0,"Parts H2O to one part fragment"],H2O)
    right.dispense(fragments.loc[0,"Parts H2O to one part fragment"],frag1dil)
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(1,frag2)
    right.dispense(1,frag2dil)
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(fragments.loc[1,"Parts H2O to one part fragment"],H2O)
    right.dispense(fragments.loc[1,"Parts H2O to one part fragment"],frag2dil)
    right.drop_tip()
    
    if number_of_rows > 2:
        right.pick_up_tip()
        right.aspirate(1,frag3)
        right.dispense(1,frag3dil)
        right.drop_tip()
        
        right.pick_up_tip()
        right.aspirate(fragments.loc[2,"Parts H2O to one part fragment"],H2O)
        right.dispense(fragments.loc[2,"Parts H2O to one part fragment"],frag3dil)
        right.drop_tip()
        
    if number_of_rows > 3:
        right.pick_up_tip()
        right.aspirate(1,frag4)
        right.dispense(1,frag4dil)
        right.drop_tip()
        
        right.pick_up_tip()
        right.aspirate(fragments.loc[3,"Parts H2O to one part fragment"],H2O)
        right.dispense(fragments.loc[3,"Parts H2O to one part fragment"],frag4dil)
        right.drop_tip()
        
    if number_of_rows > 4:
        right.pick_up_tip()
        right.aspirate(1,frag5)
        right.dispense(1,frag5dil)
        right.drop_tip()
        
        right.pick_up_tip()
        right.aspirate(fragments.loc[4,"Parts H2O to one part fragment"],H2O)
        right.dispense(fragments.loc[4,"Parts H2O to one part fragment"],frag5dil)
        right.drop_tip()
        
    if number_of_rows > 5:
        right.pick_up_tip()
        right.aspirate(1,frag6)
        right.dispense(1,frag6dil)
        right.drop_tip()
    
        right.pick_up_tip()
        right.aspirate(fragments.loc[5,"Parts H2O to one part fragment"],H2O)
        right.dispense(fragments.loc[5,"Parts H2O to one part fragment"],frag6dil)
        right.drop_tip()
    
    
    
    
    #protocol
    
    tc_mod.open_lid()
    tc_mod.set_block_temperature(4, block_max_volume=15)
    
    right.pick_up_tip()
    right.aspirate(1,frag1dil)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(1,frag2dil)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip()
    
    if number_of_rows > 2:
        right.pick_up_tip()
        right.aspirate(1,frag3dil)
        right.dispense(1,GGmix)
        right.blow_out()
        right.drop_tip()
            
    if number_of_rows > 3:
        right.pick_up_tip()
        right.aspirate(1,frag4dil)
        right.dispense(1,GGmix)
        right.blow_out()
        right.drop_tip()
        
    if number_of_rows > 4:
        right.pick_up_tip()
        right.aspirate(1,frag5dil)
        right.dispense(1,GGmix)
        right.blow_out()
        right.drop_tip()
        
    if number_of_rows > 5:
        right.pick_up_tip()
        right.aspirate(1,frag6dil)
        right.dispense(1,GGmix)
        right.blow_out()
        right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(1,T4)
    right.dispense(1,T4BSAmix)
    right.blow_out()
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(10,BSA)
    right.dispense(10,T4BSAmix)
    right.mix(5,10,T4BSAmix)
    right.blow_out()
    right.drop_tip()
    
    right.pick_up_tip()
    right.aspirate(1.65,T4BSAmix)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip
    
    right.pick_up_tip()
    right.aspirate(1,BSAI)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip
    
    right.pick_up_tip()
    right.aspirate(1,Lig)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip
    
    #Figure out amount of H2O to aspirate and dispense
    right.pick_up_tip()
    right.aspirate(1,H2O)
    right.dispense(1,GGmix)
    right.blow_out()
    right.drop_tip
    
    #pause to close cap right?
    
    tc_mod.close_lid()
    profile = [
        {'temperature': 37, 'hold_time_seconds': 180},
        {'temperature': 16, 'hold_time_seconds': 240}]
    tc_mod.execute_profile(steps=profile, repetitions=25, block_max_volume=15)
    tc_mod.set_block_temperature(4, block_max_volume=15)



In [ ]:
from opentrons import protocol_api

#Metadata is a dictionary of data that is read by the server and returned to the opentrons app. 
#give yourself credit. you are required to specify the 'apiLevel' herefrom opentrons import protocol_api
metadata = {
    'protocolName': 'ARF7 Deletions Protocol',
    'author': 'John Bryant <jbryant2@vt.edu>',
    'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants',
    'apiLevel': '2.10'
}
print(metadata)




def run(protocol: protocol_api.ProtocolContext): #for actually running the script in the robot
#will have to indent everything to be defined by the run function

#from opentrons import simulate
#protocol = simulate.get_protocol_api('2.2')
    
#labware:
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', '9')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_300ul','6')
    tiprack3 = protocol.load_labware("opentrons_96_tiprack_10ul", '5')
#tuberack1 = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap','1') #holds stock primers and templates
    watertuberack = protocol.load_labware('opentrons_10_tuberack_falcon_4x50ml_6x15ml_conical','3') #holds molec bio grad H2O
    tuberack2 = protocol.load_labware('opentrons_24_tuberack_nest_1.5ml_snapcap','2') # holds dilute primers and templates
    
    tc_mod = protocol.load_module('Thermocycler Module')
    pcrplate = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    temp_module = protocol.load_module('temperature module', 1)
    cold_tuberack = temp_module.load_labware('opentrons_24_aluminumblock_generic_2ml_screwcap', label='Temperature-Controlled Tubes')
    temp_module.set_temperature(4)
    print(temp_module.temperature)
    tc_mod.open_lid()

#########Some notes:    
#specify the order of stock primers and template in tuberack1 here:
#good place to add the pop-up window
#A1-D3 = stock primers
#D4-D5 = stock templates
#stock tubes and dilution tubes need to be set up in the same order
#as of now Q5 is in 
    
#pipettes
    right_pipette = protocol.load_instrument('p300_single','right',tip_racks=[tiprack1, tiprack2])
    left_pipette = protocol.load_instrument('p10_single','left',tip_racks = [tiprack3])
    
##################################COMMANDS####################################

#how do i set up the end of the protocol? keep the lid shut or open?
What is the code for testing the protocol again?
Talk about dilution and the .1 pipette and if there is a better way to go about this
